Analyze the astrocytes data generated by Schwab et al.

In [1]:
'''
Created on

DESCRIPTION

@author: ignacio
'''

from lib.utils import *
from lib.plot_utils import *
from lib.SCAnalyzer import SCAnalyzer
import scanpy as sc
import scvelo as scv

from func import *


import pandas..
importing path functions...
importing stat functions...
ipdb is not installed and cannot be imported...
importing misc functions..
importing done...
importing plot functions...
import scanpy
done
import others


In [2]:

revdir_by_k = {
    '20200325': '/mnt/znas/icb_zstore01/groups/bcf/workspace/SingleCellTmp/10x/projects/wurst/astrocytes_to_neuron_virus/rev7/scanpy_AnnData',
    '20200305': '/storage/groups/bcf/projects/SinglimeCell/10x/projects/wurst/astrocytes_to_neuron_virus/rev7_dep/scanpy_AnnData',
    '20200404': '/storage/groups/bcf/workspace/SingleCellTmp/10x/projects/wurst/astrocytes_to_neuron_virus/rev7/scanpy_AnnData/',
    '20200504': '/storage/groups/bcf/workspace/SingleCellTmp/10x/projects/wurst/astrocytes_to_neuron_virus/rev7/scanpy_AnnData'}


In [6]:

for date in revdir_by_k:
    if date != '20200404':
        continue

date = '20200404'

In [7]:
rcParams['figure.figsize'] = [6.4, 4.8]
revdir = revdir_by_k[date]
genes_by_t = load_data(revdir=revdir, bkpdir='../../data/bkp_anndata_astrogial_conversion')

count_depth_thr = 800
ngenes_thr = 250


output_dir = "../../data/figures/astrocytes_sc/%s" % date
if not exists(output_dir):
    makedirs(output_dir)

from matplotlib.colors import ListedColormap

for t in genes_by_t:
    adata = genes_by_t[t]
    # custom palette
    xy = adata.obsm['X_umap']
    df = adata.to_df()


    def get_dict(df, a, b):
        return pd.Series(df[b].values if b is not None else df.index,
                         index=df[a].values if a is not None else df.index).to_dict()


    SCAnalyzer.plot_qc(adata, output_dir, filename='qc_%s' % t, count_depth_thr=count_depth_thr,
                       ngenes_thr=ngenes_thr)

    clusters_dir = join(join(revdir, "cellranger", "%s/analysis/clustering" % t))
    if not exists(clusters_dir):
        continue

    if t != 'skip':
        continue

    for cluster_d2 in listdir(clusters_dir):
        print(cluster_d2)
        output_dir = join(output_dir, "%s_%s" % (t, cluster_d2))
        if not exists(output_dir):
            mkdir(output_dir)

        cluster_path = join(clusters_dir, cluster_d2, 'clusters.csv')
        print(exists(cluster_path), cluster_path)
        clusters = pd.read_csv(cluster_path)
        colors = ["#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99", "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a"]
        treatment_by_bc = get_dict(clusters, 'Barcode', 'Cluster')
        treatment_labels = sorted(list(set(clusters['Barcode'].map(treatment_by_bc))))
        palette = {k: ci for k, c, ci in zip(treatment_labels, colors, range(len(colors)))}

        scatter = plt.scatter(xy[:, 0], xy[:, 1], s=0.5, c=df.index.to_series().map(treatment_by_bc).map(palette),
                              cmap=ListedColormap(colors))

        plt.legend(handles=scatter.legend_elements()[0], labels=treatment_labels, fontsize=8)

        # for lab in treatment_labels:
        #     plt.text(xy[df.index.to_series().map(treatment_by_bc) == lab, 0].mean(),
        #              xy[df.index.to_series().map(treatment_by_bc) == lab, 1].mean(), lab, fontsize=10,
        #              c=colors[palette[lab]])
        plt.xlabel('UMAP1')
        plt.ylabel('UMAP2')
        plt.xticks([])
        plt.yticks([])
        # plt.legend(handles=scatter.legend_elements()[0],
        #            labels=[t + ", N=%i" % df.index.to_series().map(treatment_by_bc).value_counts()[t] for t in treatment_labels])
        plt.title('# cells = %i, # clusters = %s' % (df.shape[0], str(cluster_d2.split("_")[1])) if 'kmeans' in cluster_d2 else 'gclust')
        plt.savefig(join(output_dir, "umap.png"), dpi=250)
        plt.close()



done


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
findfont: Font family ['arial'] not found. Falling back to DejaVu Sans.


('saving', '../../data/figures/astrocytes_sc/20200404/qc_all.png')
saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/qc_all
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404
('saving', '../../data/figures/astrocytes_sc/20200404/qc_all.pdf')
saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/qc_all.pdf
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404
('saving', '../../data/figures/astrocytes_sc/20200404/qc_MUC17744.png')
saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/qc_MUC17744
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404
('saving', '../../data/figures/astrocytes_sc/20200404/qc_MUC17744.pdf')

In [8]:
use_hvg = True #  [True, False]:

In [10]:

# PLOT ALL CLUSTERS for both samples together
t_names = [t for t in genes_by_t]
a1, a2 = [genes_by_t[t] for t in t_names if t != 'all']
print(a1.shape, a2.shape)



# FILTER BY QC-DECIDED THRESHOLDS
a1 = a1[a1.obs['n_counts'] >= count_depth_thr, :]
a1 = a1[a1.obs['n_genes'] >= ngenes_thr, :]
a2 = a2[a2.obs['n_counts'] >= count_depth_thr, :]
a2 = a2[a2.obs['n_genes'] >= ngenes_thr, :]

batchi_by_t = {t : str(i) for i, t in enumerate(t_names)}

velodir_a1 = join(revdir.replace("scanpy_AnnData", 'velocyto'), t_names[0])
velodir_a2 = join(revdir.replace("scanpy_AnnData", 'velocyto'), t_names[1])


sc.pp.normalize_per_cell(a1, counts_per_cell_after=10000)
sc.pp.normalize_per_cell(a2, counts_per_cell_after=10000)

adata = a1.concatenate(a2)


a1_log1p = sc.pp.log1p(a1, copy=True)
a2_log1p = sc.pp.log1p(a2, copy=True)
sc.pp.highly_variable_genes(a1_log1p)
sc.pp.highly_variable_genes(a2_log1p)


var_genes = a1_log1p.var['highly_variable'] | a2_log1p.var['highly_variable']


print('all', adata.shape)
sc.pp.normalize_per_cell(adata, counts_per_cell_after=10000)
adata_log1p = sc.pp.log1p(adata, copy=True)
sc.pp.highly_variable_genes(adata_log1p)

adata.var['gene_symbol'] = list(adata.to_df().columns.values)
# Quality control - calculate QC covariates
adata.obs['n_counts'] = adata.X.sum(1)
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
adata.obs['n_genes'] = (adata.X > 0).sum(1)
mt_gene_mask = [gene.startswith('mt-') for gene in adata.var['gene_symbol']]
adata.obs['mt_frac'] = np.array(pd.DataFrame(adata.X[:, mt_gene_mask].sum(1))[0]) / adata.obs['n_counts']
adata.var['cells_per_gene'] = list(pd.DataFrame(np.sum(adata.X > 0, 0).transpose())[0])

adata_all = adata.copy()
if use_hvg:
    adata = adata[:, var_genes]
    a1 = a1[:, var_genes]
    a2 = a2[:, var_genes]



velocities_by_t = {}
load_velocities = False
if load_velocities:
    a1_velocyto = load_data_velocyto(revdir=velodir_a1, adata=a1)
    a2_velocyto = load_data_velocyto(revdir=velodir_a2, adata=a2)

    for t, v_obj in zip(t_names, [a1_velocyto, a2_velocyto]):
        velocities_by_t[t] = v_obj



print('after hvg selection %i' % use_hvg, 'all', adata.shape)

def get_dict(df, a, b):
    return pd.Series(df[b].values if b is not None else df.index,
                     index=df[a].values if a is not None else df.index).to_dict()

print('running sc.pp.neighbors...')
sc.pp.neighbors(adata)
sc.tl.umap(adata)

print('running sc.pl.umap...')
sc.tl.leiden(adata, resolution=0.05)

xy = adata.obsm['X_umap']

colors = ["red", "blue"]
treatment_labels = [t for t in genes_by_t]
palette = {k: ci for k, c, ci in zip(treatment_labels, colors, range(len(colors)))}

ax = plt.subplot()
scatter = plt.scatter(xy[:, 0], xy[:, 1], s=0.5,
                      c=list(np.repeat(0, a1.shape[0])) + list(np.repeat(1, a2.shape[0])),
                      cmap=ListedColormap(colors))

# for lab in treatment_labels:
#     plt.text(xy[df.index.to_series().map(treatment_by_bc) == lab, 0].mean(),
#              xy[df.index.to_series().map(treatment_by_bc) == lab, 1].mean(), lab, fontsize=10,
#              c=colors[palette[lab]])
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.xticks([])
plt.yticks([])
plt.legend(handles=scatter.legend_elements()[0], labels=treatment_labels)
plt.title('# cells = %i, # genes = %i' % (adata.shape[0], adata.shape[1]))
remove_all_ticks(ax)
savefig(join(output_dir, "umap_before_scanorama%s.png" % ('_hvg' if use_hvg else '')), dpi=250)
plt.close()

import scanorama
# subset the individual dataset to the same variable genes as in MNN-correct.
alldata2 = dict()
for k in t_names:
    if k == 'all':
        continue
    print(k)
    next_adata = genes_by_t[k]
    # FILTER BY QC-DECIDED THRESHOLDS
    next_adata = next_adata[next_adata .obs['n_counts'] >= count_depth_thr, :]
    next_adata = next_adata[next_adata .obs['n_genes'] >= ngenes_thr, :]
    sc.pp.normalize_per_cell(next_adata, counts_per_cell_after=10000)
    alldata2[k] = next_adata[:, var_genes]

# convert to list of AnnData objects
adatas = list(alldata2.values())
# run scanorama.integrate
scanorama = scanorama.integrate_scanpy(adatas, dimred=50,)

# make inteo one matrix.
all_s = np.concatenate(scanorama)
print(all_s.shape)

# add to the AnnData object
adata.obsm["SC"] = all_s

# tsne and umap
sc.pp.neighbors(adata, n_pcs=50, use_rep="SC")
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.05)
sc.tl.tsne(adata, n_pcs=50, use_rep="SC")

ax = plt.subplot()
sc_obj = sc.pl.tsne(adata, color="batch", title="Scanorama tsne", show=False,
                    palette=['red','blue'], size=5.0, ax=ax)
plt.xlabel('tSNE-1')
plt.ylabel('tSNE-2')
plt.xticks([])
plt.yticks([])
remove_top_n_right_ticks(ax)
plt.title('# cells = %i, # genes = %i' % (adata.shape[0], adata.shape[1]))
remove_all_ticks(ax)
savefig(join(output_dir, "tSNE_scanorama%s" % ('_hvg' if use_hvg else '')), dpi=250)
plt.close()


for add_scvelo in [False, True]:
    # make figure dimensions proportional to scVelo panel
    rcParams['figure.figsize'] = [12.0, 8.0] # [6.0, 4.5]
    for pi, label in enumerate(t_names):
        ax = plt.subplot2grid((3 if add_scvelo else 2, 3), (0, pi))
        adata_sel = adata[adata.obs['batch'] == batchi_by_t[label],:] if label != 'all' else adata

        print(label, adata_sel.shape)
        color_by_leiden = {'0': "red", '1': "blue"}
        adata_sel.uns["{var}_colors"] = adata_sel.obs['batch'].map(color_by_leiden)
        sc_obj = sc.pl.umap(adata_sel, color="batch", title="Scanorama umap", show=False,
                            size=5.0, ax=ax)
        plt.xlabel('', fontsize=8)
        plt.ylabel('UMAP2' if pi == 0 else '', fontsize=8)
        plt.xticks([])
        plt.yticks([])
        remove_top_n_right_ticks(ax)
        plt.title('#C = %i, #G = %i\n%s' % (adata_sel.shape[0], adata_sel.shape[1], label), fontsize=8)
        # if pi != 2:
        #     ax.get_legend().remove()
        remove_all_ticks(ax)
        # plt.subplots_adjust(right=0.7)
        # savefig(join(output_dir, "umap_scanorama%s_%s_batches" % ('_hvg' if use_hvg else '', label)), dpi=250)
        # plt.close()

        ax = plt.subplot2grid((3 if add_scvelo else 2, 3), (1, pi))
        colors = ["#8dd3c7", "#bc80bd", "#bebada", "#fb8072", "#80b1d3", "#fdb462", "#b3de69", "#fccde5", '#d9d9d9']
        # color_by_leiden = {'0': "#a6cee3", '1': "#1f78b4", '2': "#b2df8a", '3': "#33a02c",
        #                    '4': "#fb9a99", '5': "#e31a1c", '6': "#fdbf6f"}
        # color_by_leiden = {str(ci): c for ci, c in enumerate(colors)}
        adata_sel.uns["{var}_colors"] = adata_sel.obs['leiden'].map(color_by_leiden)
        adata_sel.uns["{var}_colors"]  ='gray'
        sc_obj = sc.pl.umap(adata_sel, color="leiden", title="Scanorama umap", show=False, size=7.5, ax=ax,
                            palette=colors)
        plt.xlabel('', fontsize=8)
        plt.ylabel('', fontsize=8)
        plt.xticks([])
        plt.yticks([])
        if pi != 2:
            ax.get_legend().remove()
        remove_top_n_right_ticks(ax)
        plt.title('#C = %i, #G = %i\n%s' % (adata_sel.shape[0], adata_sel.shape[1], label), fontsize=8)
        remove_all_ticks(ax)


        # save without velo
        # savepdf(join(output_dir, "umap_scanorama%s" % ('_hvg' if use_hvg else '')), dpi=250)

        # scvelo here
        if label == 'all':
            continue

        if not add_scvelo or not load_velocities:
            continue

        ax = plt.subplot2grid((3, 3), (2, pi))
        color_by_leiden = {'0': "#a6cee3", '1': "#1f78b4", '2': "#b2df8a", '3': "#33a02c",
                           '4': "#fb9a99", '5': "#e31a1c", '6': "#fdbf6f"}

        adata_scvelo = velocities_by_t[label]

        color_by_leiden = {'0': "#a6cee3", '1': "#1f78b4", '2': "#b2df8a", '3': "#33a02c",
                           '4': "#fb9a99", '5': "#e31a1c", '6': "#fdbf6f"}
        adata_scvelo.obs['leiden'] = np.array(adata_sel.obs['leiden'])
        adata_scvelo.uns["leiden_colors"] = pd.Series(adata_scvelo.obs['leiden']).map(color_by_leiden)


        adata_scvelo.obsm['X_umap'] = np.array(adata_sel.obsm['X_umap'])


        # adata_sel.uns["{var}_colors"] = adata_sel.obs['leiden'].map(color_by_leiden)
        # rcParams['figure.figsize'] = [3.0, 3.0]
        scv.pl.velocity_embedding_grid(adata_scvelo, arrow_length=1.0, arrow_size=.75, dpi=1000, color="leiden", size=2.5, ax=ax)

        # sc_obj = sc.pl.umap(adata_sel, color="leiden", title="Scanorama umap", show=False, size=5.0, ax=ax)
        plt.xlabel('UMAP1', fontsize=8)
        plt.ylabel('', fontsize=8)
        plt.xticks([])
        plt.yticks([])
        # if pi != 2:
        #     ax.get_legend().remove()
        remove_top_n_right_ticks(ax)
        plt.title('velocities', fontsize=8)
        remove_all_ticks(ax)


    plt.subplots_adjust(right=0.9)
    if not add_scvelo:
        savefig(join(output_dir, "umap_scanorama%s" % ('_hvg' if use_hvg else '')), dpi=250)
    else:
        savepng(join(output_dir, "umap_scanorama%s_with_scvelo" % ('_hvg' if use_hvg else '')), dpi=2000)

    plt.close()



Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


(2370, 22636) (2139, 22636)
all (3899, 22636)
after hvg selection 1 all (3899, 4274)
running sc.pp.neighbors...


findfont: Font family ['arial'] not found. Falling back to DejaVu Sans.
findfont: Font family ['arial'] not found. Falling back to DejaVu Sans.


running sc.pl.umap...
('saving', '../../data/figures/astrocytes_sc/20200404/umap_before_scanorama_hvg.png')
saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/umap_before_scanorama_hvg
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404
('saving', '../../data/figures/astrocytes_sc/20200404/umap_before_scanorama_hvg.pdf')
saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/umap_before_scanorama_hvg.pdf
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404


Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


...

('saving', '../../data/figures/astrocytes_sc/20200404/umap_scanorama_hvg.png')
saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/umap_scanorama_hvg
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404
('saving', '../../data/figures/astrocytes_sc/20200404/umap_scanorama_hvg.pdf')


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.uns` of view, copying.


saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/umap_scanorama_hvg.pdf
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404
all (3899, 4274)
MUC17744 (1875, 4274)
MUC17745 (0, 4274)


No handles with labels found to put in legend.
No handles with labels found to put in legend.


('saving', '../../data/figures/astrocytes_sc/20200404/umap_scanorama_hvg_with_scvelo.png')
saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/umap_scanorama_hvg_with_scvelo.png
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404


In [13]:
plot = False

add_rosa26 = True
pROSA_by_idx = {}
if add_rosa26:
    genes_by_t_old = load_data(revdir=revdir_by_k['20200504'])
    pROSA_by_idx['pROSA26-plus'] = DataFrameAnalyzer.get_dict(genes_by_t_old['all'].to_df(), None, 'pROSA26-plus')
    pROSA_by_idx['pROSA26-minus'] = DataFrameAnalyzer.get_dict(genes_by_t_old['all'].to_df(), None, 'pROSA26-minus')


other_genes = [mk.strip().split(" ")[0] for mk in open("../../data/markers/markers_neuron.txt")]
other_genes += ['Myt1l', 'Ascl1', 'Lmx1a', 'Slc17a7', 'Nr4a2', 'Neurl1a', 'Slc17a7', None]

other_genes += ['611_pAAV_GFP-plus', 'GFAP-Cre-plus', 'pROSA26-plus', 'pROSA26-minus']
other_genes += [c for c in adata.to_df().columns if 'olig' in c.lower()]

# to draw some marker genes using the default SC umap function well modify the batche
count_by_marker_genes = True
table = []
for count_subcluster_only in [False, True]:
    if count_by_marker_genes:
        marker_genes = [c for c in list(adata_all.to_df().columns[-6:]) if 'plus' in c or 'minus' in c] + ['pROSA26-plus', 'pROSA26-minus']

        # make figure dimensions proportional to scVelo panel
        rcParams['figure.figsize'] = [12.0, 8.0] # [12.0, 8.0] # [6.0, 4.5]
        for marker in marker_genes:
            if not '611_pAAV_GFP-plus' in marker:
                continue
            for other_gene in other_genes:
                if not other_gene in {'pROSA26-plus',}: # 'Gad2', 'Gad1'}: # {'Th', 'Slc17a7', 'GFAP-Cre-plus', '611_pAAV_GFP-plus', 'Myt1l', 'Gad1+Gad2', 'Ascl1', 'Nr4a2', 'Lmx1a'}:
                    continue
                print(marker, other_gene)

                if other_gene is None:
                    other_gene = marker

                fig = plt.figure() if plot else None
                for ri, curr_gene in enumerate([marker, other_gene]):
                    print(ri, curr_gene)
                    for pi, label in enumerate(t_names):
                        adata_sel = adata[adata.obs['batch'] == batchi_by_t[label], :] if label != 'all' else adata


                        df = adata_sel.to_df()

                        if not other_gene in df and other_gene is not None:
                            if not "+" in other_gene:
                                if other_gene != 'pROSA26-plus':
                                    if not other_gene in (adata_all[adata_all.obs['batch'] == batchi_by_t[label], :] if label != 'all' else adata_all).to_df():
                                        continue
                                    df[other_gene] = (adata_all[adata_all.obs['batch'] == batchi_by_t[label], :] if label != 'all' else adata_all).to_df()[other_gene]
                                else:
                                    q = (adata_all[adata_all.obs['batch'] == batchi_by_t[label],:] if label != 'all' else adata_all).to_df().index
                                    df[other_gene] = pd.DataFrame(q.map(pROSA_by_idx['pROSA26-plus']), index=q)

                            else:
                                for c in other_gene.split("+"):
                                    df[c] = (adata_all[adata_all.obs['batch'] == batchi_by_t[label],:] if label != 'all' else adata_all).to_df()[c]

                        # collect data by cluster
                        for leiden in set(adata_sel.obs['leiden']):
                            adata_counting = adata_sel
                            if count_subcluster_only:
                                adata_counting = adata_sel[adata_sel.obs['leiden'].isin({'2', '3'}), :].copy()

                            sel = adata_counting.to_df()[adata_sel.obs['leiden'] == leiden]


                            if other_gene is not None:
                                if other_gene in sel:
                                    other = sel[other_gene] if not '+' in other_gene else sel[[c for c in other_gene.split("+")]].max(axis=1)
                                elif other_gene == 'pROSA26-plus' and pROSA_by_idx is not None:
                                    other = pd.DataFrame((adata_all[adata_all.obs['batch'] == batchi_by_t[label],:] if label != 'all' else adata_all).to_df().index.map(pROSA_by_idx['pROSA26-plus']))
                                else:
                                    other = (adata_all[adata_all.obs['batch'] == batchi_by_t[label],:] if label != 'all' else adata_all).to_df()[[c for c in other_gene.split("+")]].max(axis=1)

                            expr = None
                            if marker in sel:
                                expr = sel[marker]
                            elif marker == 'pROSA26-plus' and pROSA_by_idx is not None:
                                expr = pd.DataFrame(sel.index.map(pROSA_by_idx['pROSA26-plus']), index=sel.index)[0]
                            else:
                                expr = (adata_all[adata_all.obs['batch'] == batchi_by_t[label], :] if label != 'all' else adata_all).to_df()[marker]
                                if count_subcluster_only:
                                    expr = (adata_all[(adata_all.obs['batch'] == batchi_by_t[label]) & (adata_all.obs['leiden'].isin({'2', '3'})), :]
                                            if label != 'all' else adata_all).to_df()[marker]

                            n_marker = ((expr > 0)).sum()
                            n_other = ((other > 0)).sum()
                            n_both = ((expr > 0) & (other > 0)).sum()
                            table.append([label, leiden, marker, n_marker, count_subcluster_only])
                            print(leiden, table[-1])
                            table.append([label, leiden, other_gene, n_other, count_subcluster_only])
                            print(leiden, table[-1])
                            table.append([label, leiden, str(other_gene) + "+" + marker, n_both])
                            print(leiden, table[-1])


                        adata_counting = adata_sel
                        if count_subcluster_only:
                            adata_counting = adata_sel[adata_sel.obs['leiden'].isin({'2', '3'}), :].copy()
                        dfc = adata_counting.to_df()
                        expr = None
                        if marker in dfc:
                            expr = dfc[marker]
                        elif marker == 'pROSA26-plus' and pROSA_by_idx is not None:
                            expr = pd.DataFrame(sel.index.map(pROSA_by_idx['pROSA26-plus']),
                                                index=sel.index)[0]
                        else:
                            expr = (adata_all[adata_all.obs['batch'] == batchi_by_t[label], :] if label != 'all' else adata_all).to_df()[marker]
                            if count_subcluster_only:
                                expr = (adata_all[(adata_all.obs['batch'] == batchi_by_t[label]) & (adata_all.obs['leiden'].isin({'2', '3'})),:]
                                        if label != 'all' else adata_all).to_df()[marker]

                        if other_gene is not None and other_gene in dfc:
                            other = dfc[other_gene] if not '+' in other_gene else dfc[[c for c in other_gene.split("+")]].max(axis=1)
                        elif other_gene == 'pROSA26-plus' and pROSA_by_idx is not None:
                            q = (adata_all[(adata_all.obs['batch'] == batchi_by_t[label]) & (adata_all.obs['leiden'].isin({'2', '3'})),:] if label != 'all' else adata_all).to_df().index
                            other = pd.DataFrame(q.map(pROSA_by_idx['pROSA26-plus']), index=q)
                        elif other_gene is not None:
                            other = (adata_all[(adata_all.obs['batch'] == batchi_by_t[label]) & (adata_all.obs['leiden'].isin({'2', '3'})),:]
                                     if label != 'all' else adata_all[(adata_all.obs['leiden'].isin({'2', '3'})),:]).to_df()[[c for c in other_gene.split("+")]].max(axis=1)
                        n_marker = ((expr > 0)).sum()
                        n_other = ((other > 0)).sum() if other_gene is not None else 0
                        n_both = ((expr > 0) & (other > 0)).sum()
                        print(n_marker, n_other, n_both)


                        adata_counting = adata_sel
                        dfc = adata_counting.to_df()
                        expr = None
                        if marker in dfc:
                            expr = dfc[marker]
                        elif marker == 'pROSA26-plus' and pROSA_by_idx is not None:
                            expr = pd.DataFrame(sel.index.map(pROSA_by_idx['pROSA26-plus']), index=sel.index)[0]
                        else:
                            expr = (adata_all[adata_all.obs['batch'] == batchi_by_t[label],
                                    :] if label != 'all' else adata_all).to_df()[marker]
                            if count_subcluster_only:
                                expr = (adata_all[(adata_all.obs['batch'] == batchi_by_t[label]) & (
                                adata_all.obs['leiden'].isin({'2', '3'})), :]
                                        if label != 'all' else adata_all).to_df()[marker]
                        if other_gene is not None:
                            if other_gene in dfc:
                                other = dfc[other_gene] if not '+' in other_gene else dfc[[c for c in other_gene.split("+")]].max(axis=1)
                            elif other_gene == 'pROSA26-plus' and pROSA_by_idx is not None:
                                q = (adata_all[adata_all.obs['batch'] == batchi_by_t[label],:] if label != 'all' else adata_all).to_df().index
                                other = pd.DataFrame(q.map(pROSA_by_idx['pROSA26-plus']), index=q)
                            else:
                                other = (adata_all[adata_all.obs['batch'] == batchi_by_t[label],:] if label != 'all' else adata_all).to_df()[[c for c in other_gene.split("+")]].max(axis=1)


                        color = pd.Series(np.where(expr > 0, 'red', '#bdbdbd'))
                        if ri != 0 and other_gene != marker:
                            color = pd.Series(np.where((expr > 0) & (other > 0), 'yellow',
                                                       np.where((expr > 0) & (other == 0), 'red',
                                                                np.where((expr == 0) & (other > 0), 'blue', '#bdbdbd'))))

                        if not plot:
                            continue


                        ax = plt.subplot2grid((2, 4), (ri, pi))
                        edgecolors_by_color = {'yellow': 'black', 'red': 'none', 'blue': 'none', '#bdbdbd': 'none'}
                        sizes_by_color = {'yellow': 2.0, 'red': 2.0, 'blue': 2.0, '#bdbdbd': 1.0}

                        
#                         SCAnalyzer.plot_umap_by_gene(adata_sel, adata_sel.to_df(), other_gene if other_gene is not None else marker,
#                                                      expr=expr, color=color, ax=ax, s=sizes_by_color, save=False, edgecolors_by_color=edgecolors_by_color,
#                                                      normed=False, fig=fig)
#                         # sc_obj = sc.pl.umap(adata_sel, color="batch", title="Scanorama umap", show=False,
#                         #                     size=5.0, ax=ax)
#                         plt.xlabel('', fontsize=8)
#                         plt.ylabel('UMAP2' if pi == 0 else '', fontsize=8)
#                         plt.xticks([])
#                         plt.yticks([])
#                         remove_top_n_right_ticks(ax)
#                         # plt.title('#C = %i, #G = %i\n%s\n%s=%i\n%s=%i\nboth=%i' %
#                         #           (adata_sel.shape[0], adata_sel.shape[1], label, marker, n_marker, str(other_gene), n_other, n_both),
#                         #           fontsize=4)
#                         # if pi != 2:
#                         #     ax.get_legend().remove()
#                         remove_all_ticks(ax)
                        # plt.subplots_adjust(right=0.7)
                        # savefig(join(output_dir, "umap_scanorama%s_%s_batches" % ('_hvg' if use_hvg else '', label)), dpi=250)
                        # plt.close()

                if plot:
                    plt.subplots_adjust(right=0.9)
                    savepdf(join(output_dir, "umap_scanorama%s_%s_%s_%s" % ('_hvg' if use_hvg else '', marker, other_gene,
                                                                            'all' if not count_subcluster_only else 'subcluster')), dpi=250)
                    plt.close()

    bar = pd.DataFrame(table, columns=['name', 'leiden', 'label', 'n', 'count.subcluster'])
    bar['k'] = bar['leiden'] + "_" + bar['label'] + "_" + bar['name']

    pi = 0
    for name, grp in bar.groupby('name'):
        grp = grp.drop_duplicates('k')
        pi += 1
        sns.heatmap(grp.pivot('label', 'leiden', 'n'))
    savepdf(join(output_dir, "counts_by_markers"))
    plt.close()

    DataFrameAnalyzer.to_tsv(bar, join(output_dir, 'barplot_n_by_leiden_n_marker_gene.tsv'))

    stop()

    # rank class genes (log1p normalized, non-batch corrected
    adata_log1p = sc.pp.log1p(adata, copy=True)

    sc.tl.rank_genes_groups(adata_log1p, 'leiden', method='t-test')
    rcParams['figure.figsize'] = [3.5, 2.0]
    sc.pl.rank_genes_groups(adata_log1p, n_genes=25, sharey=False, ncols=4, rotation='horizontal')
    savepdf(join(output_dir, 'rank_class_genes_leiden'))
    plt.close()



    # make a barplot by cluster group
    bar = adata.obs.groupby('batch')['leiden'].value_counts()
    bar.reset_index('batch')
    bar = bar.reset_index('batch')
    bar['n'] = bar['leiden']
    del bar['leiden']
    bar = bar.reset_index()
    DataFrameAnalyzer.to_tsv(bar, join(output_dir, 'barplot_n_by_leiden.tsv'))
    stop()

    plt.xlabel('cluster')
    plt.ylabel('n.cells')
    sns.barplot(data=bar, x='leiden', hue='batch', y='n')
    plt.tight_layout()
    savepdf(join(output_dir, 'barplot_n_by_leiden'))

    plt.close()

    # save barplot

    print('all', adata.shape)

    top5_per_group =  pd.DataFrame(adata_log1p.uns['rank_genes_groups']['names']).head(5)

    rcParams['figure.figsize'] = [3.5, 2.0]
    labels = []
    for leiden_cluster in [str(i) for i in range(0, 9)]:
        labels += list(top5_per_group[leiden_cluster])
    sc.pl.umap(adata, color=labels, show=False, size=15,
               edges_width=0.0, ncols=5, frameon=False)
    savefig(join(output_dir, "umap_leiden_top5_genes_per_cluster_%s" % 'hvg' if use_hvg else ''))
    plt.close()

    for leiden_cluster in [str(i) for i in range(0, 9)]:
        labels = ['leiden'] + list(top5_per_group[leiden_cluster])
        sc.pl.umap(adata, color=labels, show=False, size=15,
                   edges_width=0.0, ncols=3, frameon=False)
        savefig(join(output_dir, "umap_leiden_top5_genes_per_cluster_k%s" % leiden_cluster))
        plt.close()


    # plot the intensity of all markers genes that contain the keywords GFP or Cre in those
    adata_all.obsm['X_umap'] =  adata.obsm['X_umap']
    labels = ['leiden'] + list(adata_all.to_df().columns[-4:])
    sc.pl.umap(adata_all, color=labels, show=False, size=15,
               edges_width=0.0, ncols=3, frameon=False)
    savefig(join(output_dir, "umap_leiden_markers"))
    plt.close()









    # plot supplementary figure

    # make figure dimensions proportional to scVelo panel
    for supp_fig_label, other_genes in zip(['A', 'B', 'C', 'D', 'scVelo'],
                                           [['Ascl1', 'Myt1l', 'Gad2', 'Gad1'],
                                            ['GFAP-Cre-plus', 'Th', 'Slc17a7', None],
                                            ['Nr4a2', 'Lmx1a', None, None],
                                            ['Ppp1r1b', None, None, None],
                                            [None, None, None, None]]): # all None == scVelo
        marker_genes = adata.to_df().columns[-4:]
        rcParams['figure.figsize'] = [4.0, 2.66] # [6.0, 4.5]
        fig = plt.figure()
        for marker in marker_genes:
            if marker != '611_pAAV_GFP-plus':
                continue

            for ri, other_gene in enumerate(other_genes):
                if other_gene is None:
                    continue
                for pi, label in enumerate(t_names):
                    if label == 'all':
                        continue
                    adata_sel = adata[adata.obs['batch'] == batchi_by_t[label], :] if label != 'all' else adata

                    df = adata_sel.to_df()

                    if not other_gene in df and other_gene is not None:
                        if not "+" in other_gene:
                            if not other_gene in (adata_all[adata_all.obs['batch'] == batchi_by_t[label], :] if label != 'all' else adata_all).to_df():
                                continue
                            df[other_gene] = (adata_all[adata_all.obs['batch'] == batchi_by_t[label], :] if label != 'all' else adata_all).to_df()[other_gene]
                        else:
                            for c in other_gene.split("+"):
                                df[c] = (adata_all[adata_all.obs['batch'] == batchi_by_t[label],:] if label != 'all' else adata_all).to_df()[c]

                    # collect data by cluster
                    for leiden in set(adata.obs['leiden']):
                        sel = df[adata.obs['leiden'] == leiden]
                        expr = sel[marker]

                        if other_gene is not None:
                            other = sel[other_gene] if not '+' in other_gene else sel[[c for c in other_gene.split("+")]].min(axis=1)

                        n_marker = ((expr > 0)).sum()
                        n_other = ((other > 0)).sum()
                        n_both = ((expr > 0) & (other > 0)).sum()
                        # table.append([label, leiden, str(other_gene) + "+" + marker, n_both])

                        # print(leiden, table[-1])
                        # table.append([label, leiden, marker, n_marker])
                        # table.append([label, leiden, other_gene, n_other])

                    expr = df[marker]
                    if other_gene is not None:
                        other = df[other_gene] if not '+' in other_gene else df[[c for c in other_gene.split("+")]].min(axis=1)

                    n_marker = ((expr > 0)).sum()
                    n_other = ((other > 0)).sum() if other_gene is not None else 0
                    n_both = ((expr > 0) & (other > 0)).sum()

                    print(n_marker, n_other, n_both)


                    color = pd.Series(np.where(expr > 0, 'red', '#bdbdbd'))
                    if other_gene != marker:
                        color = pd.Series(np.where((expr > 0) & (other > 0), 'yellow',
                                                   np.where((expr > 0) & (other == 0), 'red',
                                                            np.where((expr == 0) & (other > 0), 'blue', '#bdbdbd'))))

                    ax = plt.subplot2grid((2, len(other_genes)), (pi, ri))
                    edgecolors_by_color = {'yellow': 'black', 'red': 'none', 'blue': 'none', '#bdbdbd': 'none'}
                    size_max, size_min = 1.0, .25
                    sizes_by_color = {'yellow': size_max, 'red': size_min, 'blue': size_min, '#bdbdbd': size_min}
                    SCAnalyzer.plot_umap_by_gene(adata_sel, adata_sel.to_df(), other_gene if other_gene is not None else marker,
                                                 expr=expr, color=color, ax=ax, s=sizes_by_color, save=False, edgecolors_by_color=edgecolors_by_color,
                                                 normed=False, fig=fig, size_max=size_max, size_min=size_min)
                    # sc_obj = sc.pl.umap(adata_sel, color="batch", title="Scanorama umap", show=False,
                    #                     size=5.0, ax=ax)
                    plt.xlabel('', fontsize=8)
                    plt.ylabel('UMAP2' if ri == 0 else '', fontsize=3)
                    plt.xticks([])
                    plt.yticks([])
                    remove_top_n_right_ticks(ax)
                    plt.title('%s\n%s=%i\n%s=%i\nboth=%i' %
                              (label, marker, n_marker, str(other_gene), n_other, n_both),
                              fontsize=4)
                    # if pi != 2:
                    #     ax.get_legend().remove()
                    remove_all_ticks(ax)
                    # plt.subplots_adjust(right=0.7)
                    # savefig(join(output_dir, "umap_scanorama%s_%s_batches" % ('_hvg' if use_hvg else '', label)), dpi=250)
                    # plt.close()

            plt.subplots_adjust(right=0.9, hspace=0.0)
            savepdf(join(output_dir, "umap_scanorama_suppfig%s_%s_%s" % ('_hvg' if use_hvg else '', marker, supp_fig_label)),
                    dpi=250)
            plt.close()






done
reading /storage/groups/bcf/workspace/SingleCellTmp/10x/projects/wurst/astrocytes_to_neuron_virus/rev7/scanpy_AnnData/MUC17744/filtered_feature_bc_matrix.h5ad
reading /storage/groups/bcf/workspace/SingleCellTmp/10x/projects/wurst/astrocytes_to_neuron_virus/rev7/scanpy_AnnData/MUC17745/filtered_feature_bc_matrix.h5ad
MUC17744 2370
before normalizing
MUC17744
                    n_counts  log_counts  n_genes   mt_frac
index                                                      
AAACCTGCACTACAGT-1     547.0    6.304449      138  0.736746
AAACCTGGTTAAAGAC-1    3825.0    8.249313     1392  0.016471
AAACGGGAGGAGCGTT-1    5686.0    8.645762     1945  0.075800
AAACGGGCAATGTAAG-1     687.0    6.532334      408  0.040757
AAACGGGGTTGAGTTC-1     514.0    6.242223      406  0.025292
...                      ...         ...      ...       ...
TTTGGTTCATCGTCGG-1    4834.0    8.483430     2016  0.049855
TTTGGTTGTTGCTCCT-1    3205.0    8.072468     1563  0.015289
TTTGGTTTCCTCAACC-1    5958.0    8.6

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
         Falling back to preprocessing with `sc.pp.pca` and default params.


MUC17744 4509
running sc.pp.neighbors...


         Falling back to preprocessing with `sc.pp.pca` and default params.


running sc.pl.umap...
after normalizing
MUC17744
                    n_counts  log_counts  n_genes   mt_frac leiden
index                                                             
AAACCTGCACTACAGT-1     547.0    6.304449      138  0.736746      1
AAACCTGGTTAAAGAC-1    3825.0    8.249313     1392  0.016471      0
AAACGGGAGGAGCGTT-1    5686.0    8.645762     1945  0.075800      3
AAACGGGCAATGTAAG-1     687.0    6.532334      408  0.040757      2
AAACGGGGTTGAGTTC-1     514.0    6.242223      406  0.025292      4
...                      ...         ...      ...       ...    ...
TTTGGTTCATCGTCGG-1    4834.0    8.483430     2016  0.049855      2
TTTGGTTGTTGCTCCT-1    3205.0    8.072468     1563  0.015289      3
TTTGGTTTCCTCAACC-1    5958.0    8.692491     2203  0.039778      2
TTTGTCAAGGTGATTA-1    1525.0    7.329750     1117  0.012459      4
TTTGTCAGTGATAAAC-1    7275.0    8.892200     1850  0.019656      0

[2370 rows x 5 columns]
MUC17745 2370
running sc.pp.neighbors...


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
         Falling back to preprocessing with `sc.pp.pca` and default params.


running sc.pl.umap...
after normalizing
MUC17745
                    n_counts  log_counts  n_genes   mt_frac leiden
index                                                             
AAACCTGAGGCTAGGT-1    1608.0    7.382746      904  0.029851      4
AAACCTGAGGTGCACA-1    1162.0    7.057898      866  0.025818      3
AAACCTGCAAGTCTGT-1    2962.0    7.993620     1443  0.026671      2
AAACCTGGTAAACACA-1     791.0    6.673298      438  0.309735      1
AAACCTGGTTCAACCA-1    4219.0    8.347353     2014  0.112349      2
...                      ...         ...      ...       ...    ...
TTTGGTTCACCAGATT-1    2450.0    7.803843     1447  0.036735      4
TTTGGTTCATGCTGGC-1    5957.0    8.692323     2333  0.026020      4
TTTGTCAGTAGCGCTC-1    7564.0    8.931155     2682  0.059096      1
TTTGTCAGTAGCGTAG-1     848.0    6.742881      464  0.274764      1
TTTGTCATCGCGGATC-1    1950.0    7.575585     1058  0.078462      1

[2139 rows x 5 columns]
all 2139
running sc.pp.neighbors...
running sc.pl.umap.

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


after normalizing
all
                    n_counts  log_counts  n_genes   mt_frac leiden
index                                                             
AAACCTGCACTACAGT-1     547.0    6.304449      138  0.736746      1
AAACCTGGTTAAAGAC-1    3825.0    8.249313     1392  0.016471      0
AAACGGGAGGAGCGTT-1    5686.0    8.645762     1945  0.075800      4
AAACGGGCAATGTAAG-1     687.0    6.532334      408  0.040757      2
AAACGGGGTTGAGTTC-1     514.0    6.242223      406  0.025292      3
...                      ...         ...      ...       ...    ...
TTTGGTTCACCAGATT-1    2450.0    7.803843     1447  0.036735      4
TTTGGTTCATGCTGGC-1    5957.0    8.692323     2333  0.026020      4
TTTGTCAGTAGCGCTC-1    7564.0    8.931155     2682  0.059096      1
TTTGTCAGTAGCGTAG-1     848.0    6.742881      464  0.274764      1
TTTGTCATCGCGGATC-1    1950.0    7.575585     1058  0.078462      1

[4509 rows x 5 columns]
running sc.pp.neighbors...
running sc.pl.umap...
611_pAAV_GFP-plus pROSA26-plus
0 

Exception: Data must be 1-dimensional

In [ ]:
# subclustering
subclust_outdir = join(output_dir, "subclustering")
if not exists(subclust_outdir):
    mkdir(subclust_outdir)


for add_scvelo in [False, True]:
    rcParams['figure.figsize'] = [4.0, 2.66]  # [6.0, 4.5]
    if add_scvelo:
        continue
    for scvelo_asstream in [True, False]:
        if scvelo_asstream:
            continue
        pdf_paths = []
        overwrite = 1
        for leiden_res in range(10, 110, 10):
            leiden_res /= 100.0
            print(leiden_res)
            if leiden_res != 0.3:
                continue

            pdf_path = join(subclust_outdir, "scatter_sc_neu_leidenres_%.2f.pdf" % leiden_res)
            pdf_paths.append(pdf_path)
            if exists(pdf_path) and not overwrite:
                continue

            adata_subclustering = adata[adata.obs['leiden'].isin({'2', '3'}), :]


            # tsne and umap
            sc.pp.neighbors(adata_subclustering, n_pcs=50, use_rep="SC")
            sc.tl.umap(adata_subclustering)
            sc.tl.leiden(adata_subclustering, resolution=leiden_res)

            sel_Ppp1r1b = adata_subclustering.to_df()[[c for c in adata_subclustering.to_df()
                                                           if c in {'Ppp1r1b', '611_pAAV_GFP-plus', '611_pAAV_GFP-minus',
                                                                    'GFAP-Cre-plus', 'GFAP-Cre-minus'}]]
            sel_Ppp1r1b['leiden.subcluster'] = adata_subclustering.obs['leiden']
            sel_Ppp1r1b['batch'] = adata_subclustering.obs['batch']

            sel_Ppp1r1b.to_excel('../../data/figures/astrocytes_sc/%s_leiden_0.3.xlsx' % 'Ppp1r1b')
            DataFrameAnalyzer.to_tsv_gz(sel_Ppp1r1b,
                                        '../../data/figures/astrocytes_sc/%s_leiden_0.3.tsv.gz' % 'Ppp1r1b')

            sc.tl.tsne(adata_subclustering, n_pcs=50, use_rep="SC")

            # save a subplot with the subclustering results
            # make a barplot by cluster group
            bar = adata_subclustering.obs.groupby('batch')['leiden'].value_counts()
            bar.reset_index('batch')
            bar = bar.reset_index('batch')
            bar['n'] = bar['leiden']
            del bar['leiden']
            bar = bar.reset_index()
            DataFrameAnalyzer.to_tsv(bar, join(subclust_outdir, 'barplot_n_by_leiden_%.2f.tsv' % leiden_res))

            rcParams['figure.figsize'] = [6.0, 6.0]
            fig = plt.figure(figsize=[6.0, 6.0] if not add_scvelo else [12.0, 2.66 * 3])
            for pi, label in enumerate(t_names):
                print(pi, label)
                adata_sel_subclustering = adata_subclustering[adata_subclustering.obs['batch'] == batchi_by_t[label],:] if label != 'all' else adata_subclustering
                # ax = plt.subplot2grid((3, 3), (0, pi))

                # xy = adata_sel.obsm['X_umap']
                # xy = adata[adata.obs.index.to_series().isin(adata_sel.obs.index),:].obsm['X_umap']
                # colors = ["red", "blue"]
                # treatment_labels = [t for t in genes_by_t]
                # palette = {k: ci for k, c, ci in zip(treatment_labels, colors, range(len(colors)))}
                # scatter = plt.scatter(xy[:, 0], xy[:, 1], s=0.1,
                #        c=list(adata_sel.obs['batch'].astype(int)),
                #                     cmap=ListedColormap(colors))
                # plt.xlabel('', fontsize=8)
                # plt.ylabel('UMAP2' if pi == 0 else '', fontsize=8)
                # plt.xticks([])
                # plt.yticks([])
                # remove_all_ticks(ax)


                colors = ["#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99", "#e31a1c",
                          "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a", "#ffff99", "#b15928"]

                ax = plt.subplot2grid((2, 4), (0, pi))

                color_by_leiden = {ci: c for ci, c in enumerate(colors)
                                   if ci < len(set(adata_sel_subclustering.obs['leiden']))}
                others_lab = len(set(adata_sel_subclustering.obs['leiden']))
                colors = colors[:len(color_by_leiden)] + ['gray']
                color_by_leiden[others_lab] = 'gray'

                # adata_sel.uns["leiden_colors"] = adata_sel_subclustering.obs['leiden'].map(color_by_leiden)

                leiden_by_idx = DataFrameAnalyzer.get_dict(adata_sel_subclustering.obs, None, 'leiden')

                adata_plot = adata.copy() # adata[adata.obs.index.to_series().isin(adata_sel_subclustering.obs.index),:]
                adata_plot = adata_plot[adata_plot.obs['batch'] == batchi_by_t[label], :] if label != 'all' else adata_plot
                print(label, adata_plot.shape[0])

                adata_plot.obs["leiden"] = np.where(adata_plot.obs.index.isin(set(adata_sel_subclustering.obs.index)),
                                                    adata_plot.obs.index.map(leiden_by_idx), others_lab)
                adata_plot.uns["leiden_colors"] = adata_plot.obs['leiden'].map(color_by_leiden)

                sc_obj = sc.pl.umap(adata_plot, color="leiden", title="Scanorama umap",
                                    show=False, size=5.0, ax=ax, palette=colors)
                plt.xlabel('UMAP1', fontsize=8)
                plt.ylabel('', fontsize=8)
                plt.xticks([])
                plt.yticks([])
                if pi != 2:
                    ax.get_legend().remove()
                remove_top_n_right_ticks(ax)
                plt.title('#C = %i, #G = %i\n%s' % (adata_plot.shape[0], adata_plot.shape[1], label), fontsize=8)
                remove_all_ticks(ax)

                if label == 'all':
                    continue

                if not add_scvelo:
                    continue

                ax = plt.subplot2grid((2, 4), (1, pi))

                adata_scvelo = velocities_by_t[label]

                adata_scvelo.obs['leiden'] = np.array(adata_plot.obs['leiden'])
                adata_scvelo.uns["leiden_colors"] = pd.Series(adata_scvelo.obs['leiden']).map(color_by_leiden)


                adata_scvelo.obsm['X_umap'] = np.array(adata_plot.obsm['X_umap'])


                # adata_sel.uns["{var}_colors"] = adata_sel.obs['leiden'].map(color_by_leiden)
                # rcParams['figure.figsize'] = [3.0, 3.0]
                if not scvelo_asstream:
                    scv.pl.velocity_embedding_grid(adata_scvelo, arrow_length=1.0, arrow_size=.75, dpi=2000,
                                                   color="leiden", size=2.5, ax=ax)
                else:
                    scv.pl.velocity_embedding_stream(adata_scvelo, dpi=2000, linewidth=.6,
                                                   color="leiden", size=5.0, ax=ax)

                # sc_obj = sc.pl.umap(adata_sel, color="leiden", title="Scanorama umap", show=False, size=5.0, ax=ax)
                plt.xlabel('UMAP1', fontsize=8)
                plt.ylabel('', fontsize=8)
                plt.xticks([])
                plt.yticks([])
                # if pi != 2:
                #     ax.get_legend().remove()
                remove_top_n_right_ticks(ax)
                plt.title('velocities', fontsize=8)
                remove_all_ticks(ax)


            if not add_scvelo:
                savefig(join(subclust_outdir, "scatter_sc_neu_leidenres_%.2f" % leiden_res), dpi=250)
            else:
                savepng(join(subclust_outdir, "scatter_sc_neu_leidenres_%.2f_scvelo%s" %
                             (leiden_res, '_stream' if scvelo_asstream else '')), dpi=2000)
            plt.close()

            # rank genes groups
            adata_sel_subclustering = adata_subclustering[adata_subclustering.obs['batch'] == batchi_by_t[label],:]
                if label != 'all' else adata_subclustering
            adata_rank = adata_sel_subclustering[adata_sel_subclustering.obs['leiden'] != others_lab, :].copy()

            sc.tl.rank_genes_groups(adata_rank, 'leiden', method='t-test')

            rcParams['figure.figsize'] = [2.9, 2.0]
            fig = plt.figure(figsize=[2, 2.0])
            ax2 = plt.subplot()
            sc.pl.rank_genes_groups(adata_rank, n_genes=25, sharey=False, ncols=4,ax=ax2)
            savepdf(join(subclust_outdir, "rank_class_genes_leiden_%.2f" % leiden_res))
            pdf_paths.append(join(subclust_outdir, "rank_class_genes_leiden_%.2f.pdf" % leiden_res))
            plt.close()

            # here establish a correlation heatmap between the marker genes

            sel = pd.DataFrame(adata_rank.copy().uns['rank_genes_groups']['pvals_adj'])
            sel.columns = ['padj.%s' % xi for xi in sorted(set(adata_rank.obs['leiden']))]
            genes = pd.DataFrame(adata_rank.copy().uns['rank_genes_groups']['names'])
            genes.columns = ['g.%s' % xi for xi in sorted(set(adata_rank.obs['leiden']))]
            scores = pd.DataFrame(adata_rank.copy().uns['rank_genes_groups']['scores'])
            scores.columns = ['score.%s' % xi for xi in sorted(set(adata_rank.obs['leiden']))]
            hm = pd.concat([sel, genes, scores], axis=1)

            nselect = 10

            for method in ['spearman', 'pearson']:
                if method != 'pearson':
                    continue
                rcParams['figure.figsize'] = [3.0, 3.0]  # [6.0, 4.5]
                for x1, x2 in (['0', '2'], ['3', '1']): #combinations(set(adata_rank.obs['leiden']), 2): #
                    # genes1 = list(hm.sort_values('score.%s' % x1).head(nselect)['g.%s' % x1])
                    # genes2 = list(hm.sort_values('score.%s' % x2).head(nselect)['g.%s' % x2])
                    genes1 = list(hm.sort_values('score.%s' % x1, ascending=False).head(nselect)['g.%s' % x1])
                    genes2 = list(hm.sort_values('score.%s' % x2, ascending=False).head(nselect)['g.%s' % x2])
                    print(x1, genes1)
                    print(x2, genes2)
                    print('')
                    adata_plot = adata_sel_subclustering[adata_sel_subclustering.obs['leiden'].isin({x1, x2}), :]
                    adata_plot_df = adata_plot.to_df()[genes1 + genes2]

                    sns.set(font_scale=1.0)

                    g = sns.clustermap(adata_plot_df.corr(method='pearson'), cmap='coolwarm', yticklabels=True, xticklabels=True,
                                       vmin=-.5, vmax=.5)
                    cormat = adata_plot_df.corr(method=method)

                    if x1 == '0':
                        cormat = cormat.reindex(g.data2d.index[::-1])[g.data2d.index[::-1]]
                    plt.close()

                    sns.heatmap(cormat, cmap='coolwarm', yticklabels=True, xticklabels=True,
                                vmin=-.65, vmax=.65)


                    plt.yticks(fontsize=6)
                    plt.xticks(fontsize=6)
                    plt.title('%s_corr_n%i_markergenes_%s_n_%s' % (method, nselect, x1, x2) + "\n" + str(adata_plot.obs['leiden'].value_counts()),
                              fontsize=6)
                    plt.subplots_adjust(bottom=.5, right=.6)
                    savepdf(join(output_dir, '%s_corr_n%i_markergenes_%s_n_%s' % (method, nselect, x1, x2)))
                    plt.close()

                stop()

                if method != 'pearson':
                    continue
                x1, x2, x3, x4 = ['0', '2', '3', '1']


                stop()
                genes1 = list(hm.sort_values('score.%s' % x1, ascending=False).head(nselect)['g.%s' % x1])
                genes2 = list(hm.sort_values('score.%s' % x2, ascending=False).head(nselect)['g.%s' % x2])
                genes3 = list(hm.sort_values('score.%s' % x3, ascending=False).head(nselect)['g.%s' % x3])
                genes4 = list(hm.sort_values('score.%s' % x4, ascending=False).head(nselect)['g.%s' % x4])
                print(x1, genes1)
                print(x2, genes2)
                print(x3, genes3)
                print(x4, genes4)
                print('')
                adata_plot = adata_sel_subclustering[
                             adata_sel_subclustering.obs['leiden'].isin({x1, x2, x3, x4}), :]

                custom_zscore = False
                if not custom_zscore:
                    sc.pp.scale(adata_plot)

                adata_plot_df = adata_plot.to_df()[genes1 + genes2 + genes3 + genes4]
                adata_plot_df['cell.label'] = adata_plot.obs['leiden']
                adata_plot_df['new.order'] = adata_plot_df['cell.label'].map({k: v for k, v in zip('0123', '0231')})
                adata_plot_df = adata_plot_df.sort_values('cell.label')
                color_code = dict(zip(adata_plot_df['cell.label'].unique(), colors)) # [:len(adata_plot_df['cell.label'].unique())]))
                col_colors = list(adata_plot_df['new.order'].map(color_code))
                del adata_plot_df['cell.label']
                del adata_plot_df['new.order']

                if custom_zscore:
                    local_zscore = False
                    if local_zscore:
                        for c in adata_plot_df:
                            print(c, adata_plot_df[c].mean() / adata_plot_df[c].std())
                            adata_plot_df[c] -= adata_plot_df[c].mean() / adata_plot_df[c].std()
                    else:
                        adata_plot_df -= adata_plot_df.values.mean()
                        adata_plot_df /= adata_plot_df.values.std()


                adata_plot_df = adata_plot_df.transpose()

                final_col_order = []
                col_colors = []
                leiden_by_index = {idx: r['leiden'] for idx, r in adata_plot.obs.iterrows()}
                for k in [set(ki) for ki in '0231']: # [{'0', '2'}, {'3', '1'}]: # '0231':
                    mask = adata_plot_df.columns.isin(adata_plot[adata_plot.obs['leiden'].isin(k),:].obs.index)
                    adata_plot_sel = adata_plot_df[[c for c, ki in zip(adata_plot_df, mask) if ki == True]]
                    print(k, adata_plot_sel.shape[1])
                    # reorder the columns by subclustering each of those
                    g = sns.clustermap(adata_plot_sel, cmap='coolwarm', yticklabels=False, xticklabels=False, vmin=-.5, vmax=.5)
                    final_col_order += list(g.data2d.columns)
                    col_colors += [color_code[leiden_by_index[idx]] for idx in list(g.data2d.columns)]
                    plt.close()

                sns.set(font_scale=1.0)
                vmin, vmax= -4, 4
                cmap_name = 'RdBu_r'
                g = sns.clustermap(adata_plot_df[final_col_order], cmap=cmap_name,
                               vmin=vmin, # adata_plot_df.min().min(),
                               vmax=vmax, # adata_plot_df.max().max(),
                               yticklabels=1, xticklabels=0, col_colors=col_colors,
                               col_cluster=False, cbar_kws={'label': 'Z-score'})
                g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize=6)
                plt.title('%s_corr_n%i_markergenes_%s_n_%s' % (method, nselect, x1, x2) + "\n" + str(
                    adata_plot.obs['leiden'].value_counts()),
                          fontsize=6)
                plt.subplots_adjust(bottom=.6, right=.4)

                output_dir = "../../data/figures/astrocytes_sc/%s" % date

                savepdf(join(output_dir, 'clustermap_%s' % (nselect)))
                plt.close()

                cbar_path_fig = "../../data/figures/enrichments_by_quadrant_cbar"

                # from lib.Plots.ColorBar import ColorBar
                ColorBar.make_color_bar(vmin, vmax, 'Z-score', cmap_name, join(output_dir, 'clustermap_%s_colorbar' % (nselect)),
                                        axes=[0.05, 0.80, 0.2, 0.025])

                # for mt counts
                ColorBar.make_color_bar(0, 1, 'Fraction of mitocondrial counts', cmap_name, join(output_dir, 'clustermap_%s_mtcounts' % (nselect)),
                                        axes=[0.05, 0.80, 0.2, 0.025])

            stop()


        if not add_scvelo:
            from lib.PDFParser import PDFParser
            PDFParser.concatenate(pdf_paths, join(subclust_outdir, "scatter_sc_neu_leidenres_all.pdf"))



In [14]:





from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# Scaling the data to normalize

for nk in range(3, 11):
    model = KMeans(n_clusters=nk).fit(xy)

    colors = ["#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99", "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a"]
    treatment_labels = sorted(range(nk))
    palette = {k: ci for k, c, ci in zip(treatment_labels, colors, range(len(colors)))}

    ax = plt.subplot()
    scatter = plt.scatter(xy[:, 0], xy[:, 1], s=0.5, c=model.labels_.astype(float),
                          cmap=ListedColormap(colors))
    plt.xlabel('UMAP1')
    plt.ylabel('UMAP2')
    plt.xticks([])
    plt.yticks([])
    plt.legend(handles=scatter.legend_elements()[0], labels=treatment_labels, fontsize=8)
    plt.title('# cells = %i, # genes = %i' % (adata.shape[0], adata.shape[1]))
    remove_all_ticks(ax)
    plt.savefig(join(output_dir, "umap_k%i%s.png" % (nk, '_hvg' if use_hvg else '')), dpi=250)
    plt.close()

# cluster using leiden
sc.tl.leiden(adata, resolution=0.05)
colors = ["#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99", "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a"]
cmap = ListedColormap(colors)
treatment_labels = sorted(set(adata.obs['leiden']))
palette = {k: ci for k, c, ci in zip(treatment_labels, colors, range(len(colors)))}
ax = plt.subplot()
scatter = plt.scatter(xy[:, 0], xy[:, 1], s=0.5, c=adata.obs['leiden'].map(palette), cmap=cmap)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.xticks([])
plt.yticks([])
plt.legend(handles=scatter.legend_elements()[0], labels=treatment_labels, fontsize=8)
plt.title('# cells = %i, # genes = %i' % (adata.shape[0], adata.shape[1]))
remove_all_ticks(ax)
savefig(join(output_dir, "umap_leiden%s" % ('_hvg' if use_hvg else '')), dpi=250)
plt.close()

('saving', '../../data/figures/astrocytes_sc/20200404/umap_leiden_hvg.png')
saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/umap_leiden_hvg
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404
('saving', '../../data/figures/astrocytes_sc/20200404/umap_leiden_hvg.pdf')
saved at / full path
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404/umap_leiden_hvg.pdf
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/data/figures/astrocytes_sc/20200404


In [15]:
adata

AnnData object with n_obs × n_vars = 3899 × 4274
    obs: 'n_counts', 'log_counts', 'n_genes', 'mt_frac', 'leiden', 'batch'
    var: 'gene_symbol', 'cells_per_gene-0', 'cells_per_gene-1', 'cells_per_gene'
    uns: 'neighbors', 'umap', 'leiden', 'batch_colors', '{var}_colors', 'leiden_colors'
    obsm: 'X_pca', 'X_umap', 'SC', 'X_tsne'
    obsp: 'distances', 'connectivities'

In [16]:




adata.to_df().to_csv("../../data/adata_all.tsv.gz", compression='gzip', sep='\t')
adata.write("../../data/adata_all.h5ad", compression='lzf')

In [10]:
df[other_gene] = pd.DataFrame(q.map(pROSA_by_idx['pROSA26-plus']), index=q)